# Lab 6: Strands Agents with Bedrock AgentCore Memory

This lab demonstrates how to integrate Strands Agents with Amazon Bedrock AgentCore Memory to create AI agents with persistent memory capabilities across conversations and sessions.

## Learning Objectives

By completing this lab, you will:
- Understand Bedrock AgentCore Memory capabilities and architecture
- Implement short-term and long-term memory strategies
- Store and retrieve conversation context across sessions
- Extract user preferences and semantic facts automatically
- Build memory-enabled agents for personalized experiences
- Test memory retrieval and context injection in real conversations

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of Strands Agents and Bedrock AgentCore concepts

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents and Bedrock AgentCore Python SDK:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore rich

## Demonstrating Strands Agents' Memory Limitations

By default, Strands Agents have **no built-in long-term persistent memory**. Each conversation starts fresh without access to previous interactions or learned context. Let's demonstrate this limitation before showing how AgentCore Memory solves this challenge.

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel

# Create a custom weather tool for demonstration
@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Rainy, 25°C" # dummy result for demo purpose

# Create your first agent
agent = Agent(
    model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
    system_prompt="You are a helpful assistant that provides concise responses.",
    tools=[weather],
)

agent("What did we talk about?")

## What is Bedrock AgentCore Memory?

### Short-Term Memory: 

Short-term memory captures raw interaction events, maintains immediate context, powers real-time conversations, enriches long-term memory systems, and enables building advanced contextual solutions such as multi-step task completion, in-session knowledge accumulation, and context-aware decision making.

- **Synchronous storage** - Messages saved immediately during conversations

- **Configurable retention** - Automatically expires after specified period (from 7 to 365 days)


### Long-Term Memory: 

Long-term memory stores structured information extracted from raw agent interactions, which is retained across multiple sessions. Instead of saving all raw conversation data, long-term memory preserves only the key insights such as summaries of the conversations, facts and knowledge (semantic memory), or user preferences.

The memory pipeline is an asynchronous process that runs in the background and automatically extracts insights after raw conversation/context is stored in Short Term Memory via CreateEvent. This efficiently consolidates key information without interrupting live interactions. 

![memory-pipeline.png](images/memory-pipeline.png)

## Creating AgentCore Memory

Let's create an AgentCore Memory instance with both short-term and long-term memory capabilities. This process takes **approximately 3 minutes** to set up the vector database and processing pipeline.

We'll configure three memory strategies:
- **Summary Strategy** - Generates session summaries for efficient context compression
- **User Preference Strategy** - Captures and stores user behavior patterns
- **Semantic Strategy** - Extracts and stores factual knowledge from conversations

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType
from botocore.exceptions import ClientError
import boto3

region = boto3.session.Session().region_name

memory_client = MemoryClient(region_name=region)
memory_name = "SampleAgentMemory"

try:
    memory = memory_client.create_memory_and_wait(
        name=memory_name,
        description="Memory for sample agent conversations",
        strategies=[
            {
                StrategyType.SUMMARY.value: {
                    "name": "SessionSummarizer",
                    "namespaces": ["sample-agent/summaries/{actorId}/{sessionId}"]
                }
            },
            {
                StrategyType.USER_PREFERENCE.value: {
                    "name": "UserPreferences",
                    "description": "Captures user preferences and behavior",
                    "namespaces": ["sample-agent/preferences/{actorId}"],
                }
            },
            {
                StrategyType.SEMANTIC.value: {
                    "name": "FactExtractor",
                    "description": "Stores facts from conversations",
                    "namespaces": ["sample-agent/semantic/{actorId}/"],
                }
            },
        ],
        event_expiry_days=7, # Memories expire after 7 days
    )
    memory_id = memory.get('id') # The memory_id will be used in following operations
    print(f"✅ Memory ID: {memory_id}")
except ClientError as e:
    print(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = memory_client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        print(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            memory_client.delete_memory_and_wait(memory_id=memory_id)
            print(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            print(f"Failed to clean up memory: {cleanup_error}")

## Storing Conversation Events

Now let's demonstrate how to store conversation events in AgentCore Memory. We'll create several conversation exchanges to populate both short-term and long-term memory with different types of content.

### Sample Conversation: Travel Planning Assistant

We'll simulate a comprehensive travel planning conversation where a user asks about Hong Kong activities. This conversation will demonstrate how AgentCore Memory captures:

- **Session context** (conversation flow and topics discussed)
- **User preferences** (hiking, local food experiences, family-friendly activities)
- **Semantic facts** (information about Bedrock AgentCore)


The following conversation events will populate both short-term and long-term memory with rich, diverse content for testing memory retrieval capabilities.

In [ ]:
# Simulate a conversation session with Hong Kong activities enquiry

USER_ID = "123"
SESSION_ID = "12345-67890"

memory_client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=USER_ID,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=SESSION_ID, #Unique id for a particular request/conversation.
    messages=[
        ("What activities would you suggest for Hong Kong?", "USER"),
        ("""Hong Kong offers an incredible mix of experiences! Here are some top activities to consider:

        ## Must-See Attractions
        - **Victoria Peak** - Take the historic Peak Tram for stunning city views
        - **Star Ferry** - Iconic harbor crossing between Tsim Sha Tsui and Central
        - **Symphony of Lights** - Nightly multimedia light show over Victoria Harbour
        - **Big Buddha & Po Lin Monastery** - Cable car ride to Lantau Island's giant bronze Buddha

        ## Food & Culture
        - **Dim sum** - Try traditional tea houses or modern venues
        - **Street food markets** - Temple Street Night Market, Ladies' Market
        - **Rooftop bars** - Enjoy skyline views with cocktails
        - **Traditional temples** - Man Mo Temple, Wong Tai Sin Temple

        ## Unique Experiences
        - **Tram rides** - Historic "ding-dings" through the city
        - **Dragon's Back hike** - Scenic trail with coastal views
        - **Ten Thousand Buddhas Monastery** - Atmospheric temple complex
        - **Avenue of Stars** - Waterfront promenade celebrating Hong Kong cinema

        ## Day Trips
        - **Macau** - Easy ferry ride for Portuguese culture and casinos
        - **Outlying Islands** - Lamma, Cheung Chau for beaches and seafood
        """, "ASSISTANT")
    ],
)

In [ ]:
# User preference with hiking activity and ask for Hong Kong hiking trails

USER_ID = "123"
SESSION_ID = "12345-67890"

memory_client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=USER_ID,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=SESSION_ID, #Unique id for a particular request/conversation.
    messages=[
        ("I love hiking! Can you recommend more hiking trails?", "USER"),
        ("""Excellent! Hong Kong has some of the world's best urban hiking with stunning coastal and mountain trails. Here are my top recommendations:

        ## Iconic Trails
        - **Dragon's Back** - Hong Kong Island's most famous ridge hike with panoramic ocean views
        - **Lion Rock** - Challenging climb with incredible 360° city views (symbol of Hong Kong spirit!)
        - **The Peak Circle Walk** - Easy paved trail around Victoria Peak with harbor views
        - **Violet Hill** - Great sunset spot overlooking Repulse Bay

        ## Coastal Beauties
        - **MacLehose Trail Section 2** - Sai Kung's stunning coastline and beaches
        - **Tai Long Wan** - Remote beaches accessible via scenic hiking trail
        - **Cape D'Aguilar** - Rugged coastline on Hong Kong Island's southeast tip
        - **Sharp Island** - Unique tombolo (sand bar) connecting two islands

        ## Mountain Adventures
        - **Sunset Peak** - Lantau Island's second-highest peak, famous for sunrise/sunset
        - **Tai Mo Shan** - Hong Kong's highest peak (957m) with misty mountain views
        - **Ma On Shan** - Distinctive pyramid-shaped peak with challenging ascent
        - **High Junk Peak** - Sai Kung's highest point with 270° ocean views
        """, "ASSISTANT"),   
    ],
)

In [ ]:
# Provide the background of family with kids and ask for child friendly spots

USER_ID = "123"
SESSION_ID = "12345-67890"

memory_client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=USER_ID,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=SESSION_ID, #Unique id for a particular request/conversation.
    messages=[
        ("I have kids in my family.", "USER"),
        ("""Hong Kong offers several kid-friendly hiking trails that combine adventure with manageable difficulty levels:

        **Easiest Trails for Families:**
        - **Hong Kong Trail Section 1 (The Peak Circle Walk)** - Flat, paved 3.5km loop with stunning views and playgrounds nearby (1 hour)
        - **Cheung Chau Family Trail** - Gentle paths around this car-free island with beaches and snack shops (2 hours, can be shortened)
        - **Lamma Island Family Walk** - Easy trail between villages with beaches, playgrounds, and seafood restaurants (2-3 hours)

        **Short Adventures:**
        - **Bride's Pool Nature Trail** - 1km walk to beautiful waterfalls with picnic areas (30 minutes plus play time)
        - **Lung Fu Shan Morning Trail** - Short uphill to The Peak with a visitor center and gardens (1 hour)
        - **Tai Tam Tree Walk** - Educational tree-spotting trail with reservoir views (1 hour)

        **Tips for Family Hiking:**
        - Start early to avoid midday heat
        - Pack plenty of snacks, water, and sun protection
        - Choose trails with bathroom facilities
        - Plan routes near public transportation
        - Consider bringing insect repellent

        Would you like specific recommendations based on your children's ages?        
        """, "ASSISTANT"),   
    ],
)

In [ ]:
# Interested in dim sun and follow up enquiry for dim sum restaurant recommendation

USER_ID = "123"
SESSION_ID = "12345-67890"

memory_client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=USER_ID,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=SESSION_ID, #Unique id for a particular request/conversation.
    messages=[
        ("I want to try dim sum. Could you recommend some famous dim sum restaurants in Hong Kong?", "USER"),
        ("""Absolutely! Hong Kong is the dim sum capital of the world. Here are some must-visit spots:

        ## Michelin-Starred Excellence
        - **Tim Ho Wan** - World's cheapest Michelin-starred restaurant, famous for BBQ pork buns
        - **Lung King Heen** - Four Seasons Hotel, upscale Cantonese with harbor views
        - **Duddell's** - Elegant setting in historic building, modern Cantonese cuisine

        ## Traditional Tea Houses
        - **Maxim's Palace** - Classic push-cart service in City Hall, old-school atmosphere
        - **Lin Heung Tea House** - Authentic 1920s tea house, grab dishes from roaming carts
        - **Luk Yu Tea House** - Historic 1933 establishment with traditional decor

        ## Modern Favorites
        - **Nom Wah Tea Parlor** - Trendy spot with Instagram-worthy presentations
        - **One Dim Sum** - Multiple locations, excellent value and quality
        - **Dim Sum Square** - Modern take on classics with creative presentations
        """, "ASSISTANT"),   
    ],
)

In [ ]:
# Simulate another session with technical discussion about Bedrock AgentCore

USER_ID = "123"
SESSION_ID = "67890-12345"

memory_client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=USER_ID,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=SESSION_ID, #Unique id for a particular request/conversation.
    messages=[
        ("What is Bedrock AgentCore?", "USER"),
        ("""Amazon Bedrock AgentCore enables you to deploy and operate highly effective agents securely, at scale using any framework and model. With Amazon Bedrock AgentCore, developers can accelerate AI agents into production with the scale, reliability, and security, critical to real-world deployment. AgentCore provides tools and capabilities to make agents more effective and capable, purpose-built infrastructure to securely scale agents, and controls to operate trustworthy agents. Amazon Bedrock AgentCore services are composable and work with popular open-source frameworks and any model, so you don’t have to choose between open-source flexibility and enterprise-grade security and reliability.

        Amazon Bedrock AgentCore includes the following modular Services that you can use together or independently:

        * Amazon Bedrock AgentCore Runtime
        AgentCore Runtime is a secure, serverless runtime purpose-built for deploying and scaling dynamic AI agents and tools using any open-source framework including LangGraph, CrewAI, and Strands Agents, any protocol, and any model. Runtime was built to work for agentic workloads with industry-leading extended runtime support, fast cold starts, true session isolation, built-in identity, and support for multi-modal payloads. Developers can focus on innovation while Amazon Bedrock AgentCore Runtime handles infrastructure and security—accelerating time-to-market

        * Amazon Bedrock AgentCore Identity
        AgentCore Identity provides a secure, scalable agent identity and access management capability accelerating AI agent development. It is compatible with existing identity providers, eliminating needs for user migration or rebuilding authentication flows. AgentCore Identity's helps to minimize consent fatigue with a secure token vault and allows you to build streamlined AI agent experiences. Just-enough access and secure permission delegation allow agents to securely access AWS resources and third-party tools and services.

        * Amazon Bedrock AgentCore Memory
        AgentCore Memory makes it easy for developers to build context aware agents by eliminating complex memory infrastructure management while providing full control over what the AI agent remembers. Memory provides industry-leading accuracy along with support for both short-term memory for multi-turn conversations and long-term memory that can be shared across agents and sessions.

        * Amazon Bedrock AgentCore Code Interpreter
        AgentCore Code Interpreter tool enables agents to securely execute code in isolated sandbox environments. It offers advanced configuration support and seamless integration with popular frameworks. Developers can build powerful agents for complex workflows and data analysis while meeting enterprise security requirements.

        * Amazon Bedrock AgentCore Browser
        AgentCore Browser tool provides a fast, secure, cloud-based browser runtime to enable AI agents to interact with websites at scale. It provides enterprise-grade security, comprehensive observability features, and automatically scales— all without infrastructure management overhead.

        * Amazon Bedrock AgentCore Gateway
        Amazon Bedrock AgentCore Gateway provides a secure way for agents to discover and use tools along with easy transformation of APIs, Lambda functions, and existing services into agent-compatible tools. Gateway eliminates weeks of custom code development, infrastructure provisioning, and security implementation so developers can focus on building innovative agent applications.

        * Amazon Bedrock AgentCore Observability
        AgentCore Observability helps developers trace, debug, and monitor agent performance in production through unified operational dashboards. With support for OpenTelemetry compatible telemetry and detailed visualizations of each step of the agent workflow, AgentCore enables developers to easily gain visibility into agent behavior and maintain quality standards at scale.        
        """, "ASSISTANT")
    ],
)

## Memory Retrieval and Analysis

After storing conversation events, AgentCore Memory automatically processes them to extract insights. Let's explore how to retrieve different types of memory:

### Short-term Memory Retrieval

Short-term memory provides immediate access to recent conversation history. This is useful for maintaining context within a session.

In [ ]:
USER_ID = "123"
SESSION_ID = "12345-67890"

print("Short-term memory for session ID 12345-67890 with window size = 2")
print("----------- ---------------------------------")
events = memory_client.list_events(
    memory_id=memory.get("id"),
    actor_id=USER_ID,
    session_id=SESSION_ID,
    max_results=2, # Last 2 conversations
)
events

### Long-term Memory Retrieval

Long-term memory automatically extracts and stores insights from conversations. It takes around **20-30 seconds** for Short-Term Memory processed into Long-Term Memory. 

In [ ]:
# Wait 1 minute for Long-Term Memory to be ready...
import time
time.sleep(60)

Let's explore the different types of long-term memory:

#### Summary Strategy
Retrieves session summaries that compress conversation context into key topics and outcomes.

In [ ]:
USER_ID = "123"

summaries = memory_client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"sample-agent/summaries/{USER_ID}/",
    query="What did I ask about Hong Kong?",
    top_k=3
)

i=1
for summary in summaries:
    print(f"Top {i} Result (Score: {summary['score']})")
    print("----------------")
    print(summary['content']['text'])
    i=i+1

#### User Preferences Strategy
Captures and stores user behavior patterns and preferences based on conversation history.

In [ ]:
USER_ID = "123"

preferences = memory_client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"sample-agent/preferences/{USER_ID}",
    query="Any recommendation for Japan trip?", # Semantic search for relevant user preferences based on query
    top_k=3
)

i=1
for preference in preferences:
    print(f"Top {i} Result (Score: {preference['score']})")
    print("----------------")
    print(preference['content']['text'])
    i=i+1

#### Semantic Facts Strategy
Extracts and stores factual knowledge from conversations for future reference.

In [ ]:
USER_ID = "123"

semantics = memory_client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"sample-agent/semantic/{USER_ID}",
    query="What is Bedrock AgentCore?", # Semantic search for fact and domain specific information based on query
    top_k=3
)

i=1
for semantic in semantics:
    print(f"Top {i} Result (Score: {semantic['score']})")
    print("----------------")
    print(semantic['content']['text'])
    i=i+1

List the memory records in long-term memory, taking Semantic Facts Strategy as an example:

In [ ]:
agentcore_client = boto3.client('bedrock-agentcore', region_name=region) 
response = agentcore_client.list_memory_records(memoryId=memory_id,
                                                namespace="sample-agent/semantic/123",
                                                maxResults=100)
response['memoryRecordSummaries']

## Building Memory-Enabled Agents

Now let's create a Strands Agent that automatically uses AgentCore Memory to provide personalized responses based on conversation history.

### Memory Integration Architecture

Our memory-enabled agent uses a hook-based architecture that seamlessly integrates with AgentCore Memory:

#### Pre-Processing Hook (`retrieve_user_context`)
- **Triggers**: Before processing each user query
- **Action**: Retrieves relevant context from all memory strategies
- **Result**: Injects personalized context into the agent's prompt

#### Post-Processing Hook (`save_conversation`)
- **Triggers**: After generating agent responses
- **Action**: Saves the interaction to AgentCore Memory
- **Result**: Enables continuous learning and context building

This architecture ensures that every interaction benefits from historical context while contributing to the agent's growing knowledge base.

In [10]:
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
import re
import logging

logger = logging.getLogger(__name__)
logging.getLogger().setLevel(logging.ERROR) # Set the logging level to ERROR

class LongTermMemoryHooks(HookProvider):
    """Memory hooks for long-term memory agent"""

    def __init__(
        self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str
    ):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = {
            i["type"]: i["namespaces"][0]
            for i in self.client.get_memory_strategies(self.memory_id)
        }

    def retrieve_user_context(self, event: MessageAddedEvent):
        """Retrieve user context before processing support query"""
        logger.info("Start to retrieve user context...")
        messages = event.agent.messages
        if (
            messages[-1]["role"] == "user"
            and "toolResult" not in messages[-1]["content"][0]
        ):
            user_query = messages[-1]["content"][0]["text"]

            try:
                all_context = []

                for context_type, namespace in self.namespaces.items():
                    # *** AGENTCORE MEMORY USAGE *** - Retrieve customer context from each namespace
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id, sessionId=""),
                        query=user_query,
                        top_k=3,
                    )
                    # Post-processing: Format memories into context strings
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    all_context.append(
                                        f"[{context_type.upper()}] {text}"
                                    )

                # Inject user context into the query
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0][
                        "text"
                    ] = f"User Context: {context_text}\n\n User Query: {original_text}"
                    logger.info(f"Retrieved {len(all_context)} user context items")

            except Exception as e:
                logger.error(f"Failed to retrieve user context: {e}")

    def save_conversation(self, event: AfterInvocationEvent):
        """Save user interaction after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user query and agent response
                user_query = None
                agent_response = None

                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        output_message = msg["content"][0]["text"]
                        agent_response = re.sub(r'<thinking>.*?</thinking>', '', output_message, flags=re.DOTALL).strip()

                    elif (
                        msg["role"] == "user"
                        and not user_query
                        and "toolResult" not in msg["content"][0]
                    ):
                        input_prompt = msg["content"][0]["text"]
                        user_query = re.sub(r'User Context:.*? User Query: ', '', input_prompt, flags=re.DOTALL).strip()
                        break

                if user_query and agent_response:
                    # *** AGENTCORE MEMORY USAGE *** - Save the support interaction
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[
                            (user_query, "USER"),
                            (agent_response, "ASSISTANT"),
                        ],
                    )
                    logger.info(f"Saved support interaction to memory")

        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register user support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_user_context)
        registry.add_callback(AfterInvocationEvent, self.save_conversation)
        logger.info("User support memory hooks registered")


Let build a Strands Agent with the Memory Hook to integrate with AgentCore Memory.

![memory-integration-with-agent.png](images/memory-integration-with-agent.png)

In [13]:
import uuid

from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient
import boto3

region = boto3.session.Session().region_name

memory_client = MemoryClient(region_name=region)

USER_ID = "123"
SESSION_ID = str(uuid.uuid4())
memory_hooks = LongTermMemoryHooks(memory_id, memory_client, USER_ID, SESSION_ID)

# Create a custom weather tool for demonstration
@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Rainy, 25°C" # dummy result for demo purpose

# Create the customer support agent with all 5 tools
agent = Agent(
    model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
    system_prompt="You are a helpful assistant that provides concise responses.",
    hooks=[memory_hooks], # Pass Memory Hooks
    tools=[weather],
)

### Testing Strands Agent with AgentCore Memory Integration

We'll test the capability of the Agent with memory in multiple scenarios.

1. **Conversation History Recall**: Verify the agent can access and summarize previous conversation sessions
  
2. **Context-Aware Tool Integration**: Test how stored preferences combine with real-time tool data
  
3. **Semantic Knowledge Retrieval**: Test retrieval of factual knowledge from stored conversations

In [ ]:
# Test the capability of Agent with memory to the past conversation sessions
agent("What did we talk about Hong Kong?")

In [ ]:
# Test the capability of Agent with memory and tool as context to provide more precise suggestion based on user preference and external factor
agent("Anything suggested to do in Hong Kong today based on the weather?")

In [ ]:
# Test the capability of Agent with retrieving the semantic fact in long-term memory
agent("Can AgentCore Memory handle long-term memory?")

Let's examine how Strands Agents integrate with AgentCore Memory and retrieve long term memory as context. 

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("Agent Loop Detail")
console.rule()
console.print(f"Number of Loops: {agent.event_loop_metrics.cycle_count}")

table = Table(title="Agent Messages", show_lines=True)
table.add_column("Role", style="green")
table.add_column("Text", style="magenta")
table.add_column("Tool Name", style="cyan")
table.add_column("Tool Input", style="cyan")
table.add_column("Tool Result", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## Resource Cleanup (Optional)

Clean up the AgentCore Memory resources to avoid unnecessary charges:

In [ ]:
import boto3
import os

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)

try:
    print("Deleting AgentCore Memory...")
    agentcore_control_client.delete_memory(memoryId=memory_id)
    print("✓ AgentCore Memory deletion initiated")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

## Conclusion

In this lab, you successfully:

- ✅ Created and configured Bedrock AgentCore Memory with vector storage
- ✅ Stored conversation events and built comprehensive memory datasets
- ✅ Implemented multiple memory retrieval strategies (summary, preferences, semantic facts)
- ✅ Integrated AgentCore Memory with Strands Agents for persistent conversation context
- ✅ Tested memory-enabled agents with cross-session conversation continuity
- 
## Key Benefits of AgentCore Memory\n",

-  **Persistent Context**: Maintain conversation history across multiple sessions and interactions
- **Intelligent Retrieval**: Multiple strategies for extracting relevant context (summaries, preferences, facts)
- **Scalable Storage**: Vector-based storage for efficient similarity search and retrieval
- **Seamless Integration**: Native integration with Strands Agents and other AI frameworks
- **Flexible Strategies**: Configurable memory strategies for different use cases and applications